In [1]:
import __future__

import numpy as np

from qiskit.quantum_info import SparsePauliOp

In [2]:
class FermionicOperator():

    def __init__(self, op_list: list | np.ndarray, num_qubits: int=None):
        self.op_list = op_list
        self.num_qubits = num_qubits

        self.ops_split = []

        max_idx = 0
        for op, coeff in self.op_list.items():
            op_split = op.split()
            num_ops = len(op_split)
            self.ops_split.append({
                "indices": [int(o[-1]) for o in op_split],
                "conjugation": [o[0] for o in op_split],
                "coeff": coeff
            })

            if max_idx < max(self.ops_split[-1]["indices"]):
                max_idx = max(self.ops_split[-1]["indices"])

            if num_ops % 2 != 0 or self.ops_split[-1]["conjugation"].count("+") != num_ops / 2:
                raise ValueError(f"FermionicOperator.map_operator(): {op} invalid")

        if self.num_qubits is None:
            self.num_qubits = max_idx + 1
        elif self.num_qubits < max_idx + 1:
            raise ValueError(
                f"FermionicOperator.__init__(): num_qubits ({self.num_qubits}) "
                f"is smaller than maximum operator index")

    def map_operator(self):
        mapped_op_list = []
        for i, op in enumerate(self.ops_split):
            if len(op["indices"]) == 2 and op["indices"][0] == op["indices"][1]:
                mapped_op_list.append(self._map_number_op(op))
            elif len(op["indices"]) == 2 and op["indices"][0] != op["indices"][1]:
                mapped_op_list.append(self._map_two_body_hop_op(op))
            if len(op["indices"]) == 4:
                mapped_op_list.append(self._map_four_body_hop_op(op))
            
        return mapped_op_list

    def _map_number_op(self, op):
        # p = q:
        # 2(I - Z)
        mapped_op = []
        norm = 1./2

        idx = op["indices"][0]

        mapped_op.append((
            self.num_qubits * "I",
            op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - idx - 1) * "I" + "Z" + idx * "I",
            - op["coeff"] * norm
        ))

        return SparsePauliOp.from_list(mapped_op)
    
    def _map_two_body_hop_op(self, op):
        # p > q:
        # (X - iY)_p Z_p-1 ... Z_q+1 (X + iY)_q
        # p < q:
        # - (X + iY)_q Z_q-1 ... Z_p+1 (X - iY)_p

        mapped_op = []
        norm = 1./4

        if op["conjugation"][0] == "-":
            op["indices"] = op["indices"][::-1]
            norm *= -1

        idx0 = op["indices"][0]
        idx1 = op["indices"][1]

        mapped_op.append((
            (self.num_qubits - max(idx0, idx1) - 1) * "I" + "X" + (abs(idx0 - idx1) - 1) * "Z" + "X" + min(idx0, idx1) * "I",
            op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - max(idx0, idx1) - 1) * "I" + "Y" + (abs(idx0 - idx1) - 1) * "Z" + "X" + min(idx0, idx1) * "I",
            - np.sign(idx0 - idx1) * 1.j * op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - max(idx0, idx1) - 1) * "I" + "X" + (abs(idx0 - idx1) - 1) * "Z" + "Y" + min(idx0, idx1) * "I",
            np.sign(idx0 - idx1) * 1.j * op["coeff"] * norm
        ))
        mapped_op.append((
            (self.num_qubits - max(idx0, idx1) - 1) * "I" + "Y" + (abs(idx0 - idx1) - 1) * "Z" + "Y" + min(idx0, idx1) * "I",
            op["coeff"] * norm
        ))

        return SparsePauliOp.from_list(mapped_op)


In [3]:
# test cases:
op_list = [
    # ({second_q-operator: coefficient}, num_qubits)
    ({"+_2 -_0": 1.0}, 3),  # a^dagger_2 a_0
    ({"-_1 +_0": 1.0}, 3),  # a_1 a^dagger_0
    ({"+_0 -_2": 1.0}, 3),  # a^dagger_0 a_2
    ({"-_2 +_1": 1.0}, 3),  # a_2 a^dagger_1
    ({"+_1 -_2": 1.0}, 3),  # a^dagger_1 a_2
    ({"-_1 +_2": 1.0}, 3),  # a_1 a^dagger_2
    ({"+_1 -_1": 1.0}, 3),  # a^dagger_1 a_1
]

op_mapped = []
for op in op_list:
    ferm = FermionicOperator(op[0], num_qubits=op[1])
    op_mapped.append(ferm.map_operator()[0])
    print(op_mapped[-1])

SparsePauliOp(['XZX', 'YZX', 'XZY', 'YZY'],
              coeffs=[0.25+0.j  , 0.  -0.25j, 0.  +0.25j, 0.25+0.j  ])
SparsePauliOp(['IXX', 'IYX', 'IXY', 'IYY'],
              coeffs=[-0.25+0.j  ,  0.  -0.25j,  0.  +0.25j, -0.25+0.j  ])
SparsePauliOp(['XZX', 'YZX', 'XZY', 'YZY'],
              coeffs=[0.25+0.j  , 0.  +0.25j, 0.  -0.25j, 0.25+0.j  ])
SparsePauliOp(['XXI', 'YXI', 'XYI', 'YYI'],
              coeffs=[-0.25+0.j  ,  0.  -0.25j,  0.  +0.25j, -0.25+0.j  ])
SparsePauliOp(['XXI', 'YXI', 'XYI', 'YYI'],
              coeffs=[0.25+0.j  , 0.  +0.25j, 0.  -0.25j, 0.25+0.j  ])
SparsePauliOp(['XXI', 'YXI', 'XYI', 'YYI'],
              coeffs=[-0.25+0.j  ,  0.  +0.25j,  0.  -0.25j, -0.25+0.j  ])
SparsePauliOp(['III', 'IZI'],
              coeffs=[ 0.5+0.j, -0.5+0.j])


## Using qiskit-nature

`> pip install qiskit-nature`

In [4]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.settings import QiskitNatureSettings

QiskitNatureSettings.use_pauli_sum_op = False

In [5]:
sec_q_op = FermionicOp({"+_3 -_0": 2.32})
mapped = JordanWignerMapper().map(sec_q_op)
print(mapped)

SparsePauliOp(['XZZY', 'XZZX', 'YZZY', 'YZZX'],
              coeffs=[0.  +0.58j, 0.58+0.j  , 0.58+0.j  , 0.  -0.58j])


In [6]:
sec_q_op = FermionicOp({"+_3 +_5 -_0 +_2 -_4": 1.21})
mapped = JordanWignerMapper().map(sec_q_op)
print(mapped)

SparsePauliOp(['XYXYIX', 'XXXYIX', 'XYXXIX', 'XXXXIX', 'XYXYIY', 'XXXYIY', 'XYXXIY', 'XXXXIY', 'YYXYIX', 'YXXYIX', 'YYXXIX', 'YXXXIX', 'YYXYIY', 'YXXYIY', 'YYXXIY', 'YXXXIY', 'XYYYIX', 'XXYYIX', 'XYYXIX', 'XXYXIX', 'XYYYIY', 'XXYYIY', 'XYYXIY', 'XXYXIY', 'YYYYIX', 'YXYYIX', 'YYYXIX', 'YXYXIX', 'YYYYIY', 'YXYYIY', 'YYYXIY', 'YXYXIY'],
              coeffs=[ 0.0378125+0.j       ,  0.       -0.0378125j,  0.       +0.0378125j,
  0.0378125+0.j       ,  0.       +0.0378125j,  0.0378125+0.j       ,
 -0.0378125+0.j       ,  0.       +0.0378125j,  0.       -0.0378125j,
 -0.0378125+0.j       ,  0.0378125+0.j       ,  0.       -0.0378125j,
  0.0378125+0.j       ,  0.       -0.0378125j,  0.       +0.0378125j,
  0.0378125+0.j       ,  0.       -0.0378125j, -0.0378125+0.j       ,
  0.0378125+0.j       ,  0.       -0.0378125j,  0.0378125+0.j       ,
  0.       -0.0378125j,  0.       +0.0378125j,  0.0378125+0.j       ,
 -0.0378125+0.j       ,  0.       +0.0378125j,  0.       -0.0378125j,
 -0.0378125+0

In [12]:
sec_q_op = FermionicOp(
    {
        "+_0 -_0": -1.2778530061568751,
        "+_1 -_1": -0.4482996961016379,
        "+_2 -_2": -1.2778530061568751,
        "+_3 -_3": -0.4482996961016379,
        "+_0 +_0 -_0 -_0": -0.34119476657602105,
        "+_0 +_1 -_1 -_0": -0.08950028803070331,
        "+_0 +_0 -_1 -_1": -0.08950028803070328,
        "+_0 +_1 -_0 -_1": -0.33536638915437944,
        "+_0 +_2 -_0 -_2": -0.34119476657602105,
        "+_0 +_3 -_1 -_2": -0.08950028803070331,
        "+_0 +_2 -_1 -_3": -0.08950028803070328,
        "+_0 +_3 -_0 -_3": -0.33536638915437944,
        "+_1 +_0 -_1 -_0": -0.33536638915437944,
        "+_1 +_1 -_0 -_0": -0.08950028803070331,
        "+_1 +_0 -_0 -_1": -0.08950028803070328,
        "+_1 +_1 -_1 -_1": -0.3525528160863921,
        "+_1 +_2 -_1 -_2": -0.33536638915437944,
        "+_1 +_3 -_0 -_2": -0.08950028803070331,
        "+_1 +_2 -_0 -_3": -0.08950028803070328,
        "+_1 +_3 -_1 -_3": -0.3525528160863921,
        "+_2 +_0 -_2 -_0": -0.34119476657602105,
        "+_2 +_1 -_3 -_0": -0.08950028803070331,
        "+_2 +_0 -_3 -_1": -0.08950028803070328,
        "+_2 +_1 -_2 -_1": -0.33536638915437944,
        "+_2 +_2 -_2 -_2": -0.34119476657602105,
        "+_2 +_3 -_3 -_2": -0.08950028803070331,
        "+_2 +_2 -_3 -_3": -0.08950028803070328,
        "+_2 +_3 -_2 -_3": -0.33536638915437944,
        "+_3 +_0 -_3 -_0": -0.33536638915437944,
        "+_3 +_1 -_2 -_0": -0.08950028803070331,
        "+_3 +_0 -_2 -_1": -0.08950028803070328,
        "+_3 +_1 -_3 -_1": -0.3525528160863921,
        "+_3 +_2 -_3 -_2": -0.33536638915437944,
        "+_3 +_3 -_2 -_2": -0.08950028803070331,
        "+_3 +_2 -_2 -_3": -0.08950028803070328,
        "+_3 +_3 -_3 -_3": -0.3525528160863921,
    },
    num_spin_orbitals=4,
)
mapped = JordanWignerMapper().map(sec_q_op)
print(mapped)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'XXYY', 'YYYY', 'XXXX', 'YYXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.79804642+0.j,  0.17771287+0.j, -0.24274281+0.j,  0.17771287+0.j,
 -0.24274281+0.j,  0.12293305+0.j,  0.17059738+0.j,  0.04475014+0.j,
  0.04475014+0.j,  0.04475014+0.j,  0.04475014+0.j,  0.16768319+0.j,
  0.16768319+0.j,  0.17627641+0.j,  0.12293305+0.j])


### Comparing to our impementation:

In [13]:
op_mapped_test = []
for op in op_list:
    op_mapped_test.append(
        JordanWignerMapper().map(FermionicOp(op[0], num_spin_orbitals=op[1]))
    )
    print(op_mapped_test[-1])

SparsePauliOp(['XZY', 'XZX', 'YZY', 'YZX'],
              coeffs=[0.  +0.25j, 0.25+0.j  , 0.25+0.j  , 0.  -0.25j])
SparsePauliOp(['IXY', 'IXX', 'IYY', 'IYX'],
              coeffs=[ 0.  +0.25j, -0.25+0.j  , -0.25+0.j  ,  0.  -0.25j])
SparsePauliOp(['XZY', 'YZY', 'XZX', 'YZX'],
              coeffs=[0.  -0.25j, 0.25+0.j  , 0.25+0.j  , 0.  +0.25j])
SparsePauliOp(['XYI', 'XXI', 'YYI', 'YXI'],
              coeffs=[ 0.  +0.25j, -0.25+0.j  , -0.25+0.j  ,  0.  -0.25j])
SparsePauliOp(['XYI', 'YYI', 'XXI', 'YXI'],
              coeffs=[0.  -0.25j, 0.25+0.j  , 0.25+0.j  , 0.  +0.25j])
SparsePauliOp(['XYI', 'YYI', 'XXI', 'YXI'],
              coeffs=[ 0.  -0.25j, -0.25+0.j  , -0.25+0.j  ,  0.  +0.25j])
SparsePauliOp(['III', 'IZI'],
              coeffs=[ 0.5+0.j, -0.5+0.j])


In [14]:
for o in range(len(op_mapped)):
    print(op_mapped[o].sort() == op_mapped_test[o].sort())

True
True
True
True
True
True
True
